**Author:** A.S. Grm (aleksander.grm@fpp.uni-lj.si)

**Date:** 2024

<hr>

# Simple method of Calculating Astronomical Position with Two Intercept

This method calculates the astronomical position when the observer's position is known. It involves only two celestial bodies!

Given our assumed position **AP** ($\varphi_\text{AP},\lambda_\text{AP}$) and the height difference $\Delta h_i$ and azimuth $\omega_i$ for two celestial bodies, we can determine the calculated position using the following approach.

This approach is based on 2D geometry and looks for the intersection of two lines $y=k\:x +n$. The procedure for the solution is very simple, but all navigational data needs to be converted to mathematical data for computational purposes.

First, navigational azimuth $\omega$ needs to be converted into mathematical angle $\alpha$

$$\alpha = 
\begin{cases}
90^\circ - \omega &; \omega \in [0^\circ,90^\circ] \\
90^\circ + \omega &; \omega \in (90^\circ,270^\circ] \\
\omega - 450^\circ &; \omega \in (270^\circ,360^\circ)
\end{cases}$$

Here, it must be emphasised, if the intercept $\Delta H < 0$ then $\omega$ must be rotated for $180^\circ$, $\omega = \omega + 180^\circ$. It must be checked if $\omega > 360^\circ$ then it must be reduced $\omega = \omega - 360^\circ$.

Using the intercept $\Delta H$ and azimuth $\omega$ it is possible to obtain a point on $\text{Lop}$ with coordinates

$$\begin{align*}
\Delta x_i & = \Delta H_i \: \cos \alpha_i \\
\Delta y_i & = \Delta H_i \: \sin \alpha_i
\end{align*}$$

where $i=1,2$ (both CB). Data can be set for $\text{Lop}$ as a line $y = k \:x + n$. First, azimuth $\omega$ must be rotated for $90^\circ$, $\beta = \omega + 90^\circ$. It must be checked if $\beta > 360^\circ$ then it must be reduced $\beta = \beta - 360^\circ$. the angle $\beta$ must be converted to mathematical angle $\alpha_\text{Lop}$. Let us find the line parameters $k_i$ and $n_i$

$$\begin{align*}
k_i & = \tan \alpha_\text{Lop}\\
n_i & = \Delta y_i - k_i \: \Delta x_i
\end{align*}$$

the intersection point is found as

$$\begin{align*}
x_p & = \frac{n_1 - n_2}{k_2 - k_1}\\
y_p & = k_1 \: x_p + n_1. 
\end{align*}$$

What remains is to convert it to a navigational position. As $\Delta H$ is in arc minutes, the $x_p$ and  $y_p$ are in arc minutes, so the observer position is

$$\begin{align*}
\varphi_p & = \varphi_\text{AP} + y_p\\
\lambda_p & = \lambda_\text{AP} + x_p \: \cos \varphi_p
\end{align*}$$

In $x$ direction, so for longitude direction, the coordinate $x_p$ must be reduced for the Mercator projection of middle latitude, as in this case it is $\varphi_p$.
<hr>

In [ ]:
import os, sys

# add custom modules and astro data path 
pp = '../nav_tools/'
sys.path.append(pp)

In [ ]:
import math as mat
import numpy as np
import matplotlib.pyplot as mpl
#mpl.rcParams['text.usetex'] = True
#mpl.rcParams.update({'font.size': 7})
import mpl_toolkits.basemap as bmap


import celestialdata as cdata
import navigationalstars as ns
import navtools as nt

# Line directional coefficient for vertical asymptote
kVA = 1e10

In [ ]:
# Convert navigational angle to mathematical
def nav2mat(w):
    
    if w < 90:
        a = 90 - w
    elif (w >= 90) and (w < 270):
        a = 90 - w
    else:
        a = 450 - w

    return a

In [ ]:
# Get Lop line data:
def getLopLineData(cb):

    dh = np.abs(cb[0])
    # rotate azimut for 180 if dh < 0
    if cb[0] > 0:
        wCB = cb[1]
    else:
        wCB = cb[1] + 180
        if wCB > 360: wCB = wCB - 360
    #print('wCB:', wCB)

    aCB = nav2mat(wCB)
    dx = dh * mat.cos(aCB*mat.pi/180)
    dy = dh * mat.sin(aCB*mat.pi/180)

    wLop = wCB + 90
    if wLop > 360: wLop = wLop - 360
    #print('wLop:', wLop)

    aLop = nav2mat(wLop)

    if (wLop == 90) or (wLop == 270):
        k = 0
        n = dy
    elif (wLop == 0) or (wLop == 180) or (wLop == 360):
        k = 1e10
        n = dx
    else:
        k = mat.tan(aLop*mat.pi/180)
        n = dy - k * dx

    return [k,n]

In [ ]:
# Find Lop line intersection point
def findIntersectionPoint(l1, l2):

    [k1, n1] = l1
    [k2, n2] = l2

    if k1 == kVA and k2 != kVA:
        x = n1
        y = k2*x + n2
    elif k2 == kVA and k1 != kVA:
        x = n2
        y = k1*x + n1
    elif (k2-k1) == 0:
        #print(' -> Error: lines are parallel')
        x = 0
        y = 0
    else:
        x = (n1 - n2)/(k2 - k1)
        y = k1*x + n1

    return [x,y]

In [ ]:
# Calculate the observer position
def calculatePos(AP,cb1,cb2):

    l1 = getLopLineData(cb1)
    l2 = getLopLineData(cb2)

    [x,y] = findIntersectionPoint(l1,l2) 
    #print('x:', x, ', y:', y)

    latP = AP[0] + y/60
    longP = AP[1] + x*mat.cos(latP*mat.pi/180)/60

    return [latP, longP]

In [ ]:
# Finds Lop bounding-box intersection points

def getLopPlotPoints(lCB):

    limBB = 90
    lT = [0, limBB]     # top BB
    lB = [0, -limBB]    # bottom BB
    lL = [kVA, -limBB]  # left BB
    lR = [kVA, limBB]   # right BB

    ipv = []
    ipv.append([findIntersectionPoint(lCB,lT),'H'])
    ipv.append([findIntersectionPoint(lCB,lB),'H'])
    ipv.append([findIntersectionPoint(lCB,lL),'V'])
    ipv.append([findIntersectionPoint(lCB,lR),'V'])

    ipBB = []
    for ip in ipv:
        x = ip[0][0]
        y = ip[0][1]
        s = ip[1]
        
        if (x != 0) and (y != 0):
            if s == 'H':
                if (x > -limBB) and (x <= limBB):
                    ipBB.append([x,y])
            elif s == 'V':
                if (y > -limBB) and (y <= limBB):
                    ipBB.append([x,y])
                    

    ipBB = np.transpose(np.array(ipBB))/60 # convert to arc degrees
    #print('ipBB:', np.transpose(ipBB)*60)
    
    return ipBB                  

In [ ]:
def plotData(AP, lop1, lop2):

    fi0 = AP[0]
    la0 = AP[1]
    dp = 1.5
    dN = 9

    fiLop1 = lop1[1] + fi0  
    laLop1 = lop1[0]*mat.cos(fi0*mat.pi/180) + la0 
    fiLop2 = lop2[1] + fi0  
    laLop2 = lop2[0]*mat.cos(fi0*mat.pi/180) + la0 
    
    map = bmap.Basemap(projection='merc',
               llcrnrlat=fi0 - dp - 0.1, # lower left corner
               llcrnrlon=la0 - dp - 0.1,
               urcrnrlat=fi0 + dp + 0.1,  # upper right corner
               urcrnrlon=la0 + dp + 0.1,
               resolution='c')

    parallels = fi0 + np.linspace(-dp,dp,dN*2+1)
    meridians = la0 + np.linspace(-dp,dp,dN*2+1)
    
    map.drawparallels(parallels, linewidth=0.25, labels=[False,False,False,False])
    map.drawparallels([fi0], linewidth=0.4, color='black')
    
    map.drawmeridians(meridians, linewidth=0.25, labels=[False,False,False,False])
    map.drawmeridians([la0], linewidth=0.4, color='black')

    map.plot(la0, fi0, marker='.', color='blue', linewidth=0.25, markersize=2, latlon=True)
    
    map.plot(laLop1, fiLop1, color='red', linewidth=0.25, latlon=True)
    map.plot(laLop2, fiLop2, color='green', linewidth=0.25, latlon=True)

    #title = 'AP: lat={:}, long={:}'.format(nt.prettyPrintLat(fi0),nt.prettyPrintLat(la0))
    mpl.title('Plotting sheet in Mercator projection')
    mpl.savefig('plotting_sheet.pdf')
    mpl.show()

In [ ]:
# Main test

AP = [[30,30,'N'],[50,30,'W']] # apparent position
cb1 = [-30, 20]  # intercept and azimuth for CB1
cb2 = [-30, 120] # intercept and azimuth for CB2

# convert navigational position to decimal degrees
OP = [nt.nav2dd(AP[0]), nt.nav2dd(AP[1])]

# find all LOP line data
lCB1 = getLopLineData(cb1)
lop1 = getLopPlotPoints(lCB1)
lCB2 = getLopLineData(cb2)
lop2 = getLopPlotPoints(lCB2)

# calculate position with two LOPs
[lat, long] = calculatePos(OP,cb1,cb2)

print()
print('Celestial body 1:')
print('  -> dH =  {:.1f} min'.format(cb1[0]))
print('  ->  w =  {:.1f} deg'.format(cb1[1]))
print()
print('Celestial body 2:')
print('  -> dH =  {:.1f} min'.format(cb2[0]))
print('  ->  w =  {:.1f} deg'.format(cb2[1]))
print()
print('Apparent position:')
print('  -> fi =  {:}'.format(nt.prettyPrintLat(OP[0])))
print('  -> la = {:}'.format(nt.prettyPrintLong(OP[1])))
print()
print('Calculated position:')
print('  -> fi =  {:}'.format(nt.prettyPrintLat(lat)))
print('  -> la = {:}'.format(nt.prettyPrintLong(long)))

In [ ]:
# Plot data

plotData(OP, lop1, lop2)